In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xml.dom import minidom
from Layer import Layer

In [5]:
# Find root of the xml file

# Define the path to snowpits folder
folder_path = "snowpits_200_MT" # The latest 200 snow pits from MT

# Create a list of all CAAML files in the folder
caaml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')] # List of all CAAML files in the folder
file = caaml_files[0]

file_path = folder_path + '/' + file
doc = minidom.parse(file_path)
root = doc.documentElement

print("ROOT NODE:")
print(root.nodeName)

ROOT NODE:
caaml:SnowProfile


In [24]:
# Layer 1 Nodes

print("root children:")

for child in root.childNodes:
    if child.nodeName != '#text':
        print(child.nodeName)


root children:
caaml:metaData
caaml:timeRef
caaml:srcRef
caaml:locRef
caaml:snowProfileResultsOf
caaml:application
caaml:applicationVersion


In [11]:
# Layer 2: Children of metaData node

print("metaData Children:")

metaData_list = root.getElementsByTagName('caaml:metaData')
for child in metaData_list:
    for subchild in child.childNodes:
        if subchild.nodeName != '#text':
            print(subchild.nodeName)


MetaData Children:
caaml:comment
caaml:customData
caaml:customData
caaml:comment
caaml:comment


In [22]:
# Layer 3:  subchildren of metaData node
metaData_list = root.getElementsByTagName('caaml:metaData')
for child in metaData_list:
    for subchild in child.childNodes:
        for subsubchild in subchild.childNodes:
            print(subsubchild.nodeName)
            print(subsubchild.nodeValue)


#text
Lazar and Heath
#text
new snow
#text
Q1 planar


In [38]:
# get string from comment node
metaData_list = root.getElementsByTagName('caaml:metaData')
for child in metaData_list:
    for subchild in child.childNodes:
        for subsubchild in subchild.childNodes:
            print(subsubchild.nodeValue)


Lazar and Heath
new snow
Q1 planar


In [12]:
# Layer 2: Children of timeRef node

print("timeRef Children:")

timeRef_list = root.getElementsByTagName('caaml:timeRef')
for child in timeRef_list:
    for subchild in child.childNodes:
        if subchild.nodeName != '#text':
            print(subchild.nodeName)

timeRef Children:
caaml:recordTime
caaml:dateTimeReport
caaml:dateTimeLastEdit


In [52]:
# Get Date and Time from timeRef node
timeRef_list = root.getElementsByTagName('caaml:dateTimeReport')
dateTime=timeRef_list[0].firstChild.nodeValue
print(dateTime)

dateTime=dateTime.split('T')
print(dateTime)


2024-11-11T05:28:11-09:00
['2024-11-11', '05:28:11-09:00']


In [27]:
# Layer 3:  subchildren of timeRef node
timeRef_list = root.getElementsByTagName('caaml:timeRef')
for child in timeRef_list:
    print(child.nodeName)
    for subchild in child.childNodes:
        print(subchild.nodeName)
        for subsubchild in subchild.childNodes:
            print(subsubchild.nodeName)
            print(subsubchild.nodeValue)

caaml:timeRef
#text
caaml:recordTime
#text

      
caaml:TimeInstant
None
#text

    
#text
caaml:dateTimeReport
#text
2024-11-11T05:28:11-09:00
#text
caaml:dateTimeLastEdit
#text
2024-11-11T05:28:45-09:00
#text


In [13]:
# Layer 2: Children of srcRef node

print("srcRef Children:")

srcRef_list = root.getElementsByTagName('caaml:srcRef')
for child in srcRef_list:
    for subchild in child.childNodes:
        if subchild.nodeName != '#text':
            print(subchild.nodeName)

srcRef Children:
caaml:Operation


In [14]:
# Layer 2: Children of locRef node

print("locRef Children:")

locRef_list = root.getElementsByTagName('caaml:locRef')
for child in locRef_list:
    for subchild in child.childNodes:
        if subchild.nodeName != '#text':
            print(subchild.nodeName)

locRef Children:
caaml:name
caaml:obsPointSubType
caaml:validElevation
caaml:validAspect
caaml:validSlopeAngle
caaml:pointLocation
caaml:country
caaml:region


In [108]:
# elevation
elevation_list = root.getElementsByTagName('caaml:ElevationPosition')

elevation=elevation_list[0].childNodes[1].firstChild.nodeValue if elevation_list[0].childNodes[1].firstChild else None
elevation_unit=elevation_list[0].getAttribute('uom') if elevation_list[0].hasAttribute('uom') else None

print(elevation)
print(elevation_unit)




2532
m


In [120]:
aspect_list = root.getElementsByTagName('caaml:AspectPosition')
aspect_value=aspect_list[0].childNodes[1].firstChild.nodeValue if aspect_list[0].childNodes[1].firstChild else None



In [121]:
# Slope Angle
slopeAngle_list = root.getElementsByTagName('caaml:SlopeAnglePosition')

slopeAngle_val=slopeAngle_list[0].childNodes[1].firstChild.nodeValue if slopeAngle_list[0].childNodes[1].firstChild else None
slopeAngle_unit=slopeAngle_list[0].getAttribute('uom') if slopeAngle_list[0].hasAttribute('uom') else None

slopeAngle=[slopeAngle_val, slopeAngle_unit]

print(slopeAngle)
print(slopeAngle_unit)


31
deg


In [141]:
# latitude
latLong_list = root.getElementsByTagName('caaml:pointLocation')

lat_long=latLong_list[0].childNodes[1].childNodes[1].firstChild.nodeValue

lat_long=lat_long.split(' ')

print(lat_long)


45.8198520 -110.9286980
['45.8198520', '-110.9286980']


In [147]:
# Country
country_list = root.getElementsByTagName('caaml:country')
country=country_list[0].firstChild.nodeValue

print(country)

region_list = root.getElementsByTagName('caaml:region')
region=region_list[0].firstChild.nodeValue

print(region)






US
MT


In [149]:
test_str = """
test1
test2
"""

print(test_str)





test1
test2



In [15]:
# Layer 2: Children of snowProfileResultsOf node

print("snowProfileResultsOf Children:")

snowProfileResultsOf_list = root.getElementsByTagName('caaml:snowProfileResultsOf')
for child in snowProfileResultsOf_list:
    for subchild in child.childNodes:
        if subchild.nodeName != '#text':
            print(subchild.nodeName)

snowProfileResultsOf Children:
caaml:SnowProfileMeasurements


In [16]:
# Layer 2: Children of application node

print("application Children:")

application_list = root.getElementsByTagName('caaml:application')
for child in application_list:
    for subchild in child.childNodes:
        if subchild.nodeName != '#text':
            print(subchild.nodeName)

application Children:


In [17]:
# Layer 2: Children of applicationVersion node

print("applicationVersion Children:")

applicationVersion_list = root.getElementsByTagName('caaml:applicationVersion')
for child in applicationVersion_list:
    for subchild in child.childNodes:
        if subchild.nodeName != '#text':
            print(subchild.nodeName)

applicationVersion Children:


## Try other parser

In [6]:
import xml.etree.ElementTree as ET

common_tag = '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}'
root = ET.parse(file_path).getroot()


In [18]:
hs_tag = common_tag + 'profileDepth'
        
try:
    hs = next(root.iter(hs_tag), None).text
    hs_units = next(root.iter(hs_tag), None).get('uom')
    HS = [float(hs), hs_units]
except AttributeError:
    HS = None


{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}profileDepth
[68.0, 'cm']


In [41]:
# pitID
#  <caaml:locRef gml:id="location-nid-66387">

pitID_tag = common_tag + 'locRef'
gml_tag = '{http://www.opengis.net/gml}id'

try:
    pitID_str = next(root.iter(pitID_tag), None).attrib[gml_tag]
    pitID = pitID_str.split('-')[-1]
except AttributeError:
    pitID = None

print(pitID)

66387


In [45]:
dateTime_tag = common_tag + 'timePosition'

try:
    dt = next(root.iter(dateTime_tag), None).text
except AttributeError:
    dt = None

date = dt.split('T')[0] if dt is not None else None

print(dt)
print(date)


2024-11-06T11:16:00
2024-11-06


In [46]:
#<caaml:Operation gml:id="SnowPilot-Group-68">
 #     <caaml:name>Bridger Bowl Ski Patrol</caaml:name>
  #    <caaml:contactPerson gml:id="SnowPilot-User-2900">
   #     <caaml:name>BB Ski Patrol</caaml:name>



{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}


In [53]:
POS_LAT_LONG = '{http://www.opengis.net/gml}pos'
locations_params_tags = [common_tag + 'ElevationPosition', 
                                common_tag + 'AspectPosition', 
                                common_tag + 'SlopeAnglePosition']
name_front_trim = len(common_tag)
name_back_trim = -len('Position')

location = {}

try: 
    loc = next(root.iter(POS_LAT_LONG), None).text
    location['lat'], location['long'] = map(float, loc.split(' '))
except AttributeError:
    location = None
    
position_params = [t for t in root.iter() if t.tag in locations_params_tags]
print(position_params)
for tp in position_params:
    print(tp.tag[name_front_trim: name_back_trim])
    location[tp.tag[name_front_trim: name_back_trim]] = tp.find(common_tag + 'position').text

print(location)

[<Element '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}ElevationPosition' at 0x00000275E5116E80>, <Element '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}AspectPosition' at 0x00000275E5116F70>, <Element '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}SlopeAnglePosition' at 0x00000275E5117060>]
Elevation
Aspect
SlopeAngle
{'lat': 45.819852, 'long': -110.928698, 'Elevation': '2532', 'Aspect': 'E', 'SlopeAngle': '31'}
